# Aprendizaje automático relacional

#### Fernando Jesús Fernández Gallardo
#### Carmen Galván López

## Preparación

#### Imports y variables globales

In [80]:
import pandas
import numpy
import sklearn
import networkx as nx
from sklearn import preprocessing, model_selection, naive_bayes, neighbors
from sklearn.model_selection import ShuffleSplit, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

semilla = 86

#### Lectura y procesamiento inicial de los datos brutos

In [81]:
#Leemos los archivos
aristas = pandas.read_csv('data/political-books-nodes.csv')
vertices = pandas.read_csv('data/political-books-edges.csv')

#Borramos la columna ID
del(aristas['Id'])

#Mostramos las primeras 35 filas
aristas.head(35)

,Label,political_ideology
0,1000 Years for Revenge,neutral
1,Bush vs. the Beltway,conservative
2,Charlie Wilson's War,conservative
3,Losing Bin Laden,conservative
4,Sleeping With the Devil,neutral
5,The Man Who Warned America,conservative
6,Why America Slept,neutral
7,Ghost Wars,neutral
8,A National Party No More,conservative
9,Bush Country,conservative


#### Selección y validación de los datos brutos

In [82]:
"""
Comprobamos que el dataset es válido verificando que no existen duplicados
"""
if len(aristas) != len(set(aristas['Label'])):
    raise ValueError("El dataset no es válido ya que contiene duplicados")
"""
La mejor forma de identificar cada uno de los elementos que forma parte
del conjunto de entrenamiento es el nombre del propio libro (que en el dataset
se llama 'Label') en vez del ID o cualquier otro tipo de indentificador más
complejo. De esta forma, también es más fácil identificar elementos duplicados
(si los hubiera)
"""
atributos = aristas['Label']
"""
Nuestro objetivo es predecir la ideología política del autor basándonos en
sus obras, por lo que el objetivo que perseguimos en nuestro modelo
es el de la ideología política
"""
objetivos = aristas['political_ideology']

## Inicio del entrenamiento
#### Codificación del objetivo

In [93]:

"""
Para poder trabajar con los datos que tenemos, necesitamos convertirlos en un formato que sklearn pueda "entender".
Debemos de hacer que nuestros datos "planos" sean para sklearn objetos "comparables", dependiendo del tipo de
ordenación que nosotros veamos más apropiada para el método en cuestión
(de una manera similar hacemos en Java cuando implementamos la interfaz 'Comparable' y el método compareTo)

El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja
con una lista o array unidimensional de sus valores y admite cadenas

"""
# Codificadores
codificador_atributos = preprocessing.LabelEncoder()
codificador_objetivos = preprocessing.LabelEncoder()
# Datos codificados
atributos_codificados = codificador_atributos.fit_transform(atributos)
objetivos_codificados = codificador_objetivos.fit_transform(objetivos)

#### División en conjunto de entrenamiento y conjunto de prueba

Partimos el atributo y el objetivo en dos, de entrenamiento y de prueba

In [98]:
(atributos_entrenamiento,
 atributos_prueba,
 objetivos_entrenamiento,
 objetivos_prueba) = model_selection.train_test_split(
        atributos_codificados,
        objetivos_codificados,
        # Valor de la semilla aleatoria para que el muestreo sea reproducible a pesar de ser aleatorio
        random_state=semilla,
        test_size=.33,
        stratify=objetivos_codificados
)

cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

## KNN no relacional

In [99]:
def encontrar_mejor_k(atributos, objetivo, k_range, cv=5):
    puntajes_por_k = []

    # Convertir los atributos en un array bidimensional con una sola columna
    atributos = np.array(atributos).reshape(-1, 1)

    for k in k_range:
        # Crear clasificador KNN con el valor actual de k
        knn = KNeighborsClassifier(n_neighbors=k)

        # Suprimir los warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            # Realizar validación cruzada y obtener los puntajes
            puntajes = cross_val_score(knn, atributos, objetivo, cv=cv)

        # Calcular el puntaje medio de validación cruzada
        puntaje_medio = puntajes.mean()

        # Almacenar el puntaje correspondiente al valor de k
        puntajes_por_k.append((k, puntaje_medio))

    return puntajes_por_k

puntajes_por_k = encontrar_mejor_k(atributos_entrenamiento, objetivos_entrenamiento, k_range=list(range(1, 21)), cv=5)

# Imprimir los puntajes para cada valor de k
for k, puntaje in puntajes_por_k:
    print(f"Puntaje para k={k}: {puntaje}")
    

# Obtener el número de vecinos con el puntaje más alto
mejor_k, mejor_puntaje = max(puntajes_por_k, key=lambda x: x[1])

# Imprimir el número de vecinos con el puntaje más alto
print(f"El número de vecinos con el puntaje más alto es: {mejor_k}")

Puntaje para k=1: 0.2714285714285714
Puntaje para k=2: 0.32857142857142857
Puntaje para k=3: 0.2571428571428571
Puntaje para k=4: 0.2714285714285714
Puntaje para k=5: 0.3571428571428571
Puntaje para k=6: 0.38571428571428573
Puntaje para k=7: 0.4
Puntaje para k=8: 0.3285714285714286
Puntaje para k=9: 0.3571428571428571
Puntaje para k=10: 0.2857142857142857
Puntaje para k=11: 0.3571428571428571
Puntaje para k=12: 0.32857142857142857
Puntaje para k=13: 0.32857142857142857
Puntaje para k=14: 0.37142857142857144
Puntaje para k=15: 0.34285714285714286
Puntaje para k=16: 0.37142857142857144
Puntaje para k=17: 0.3571428571428571
Puntaje para k=18: 0.3857142857142857
Puntaje para k=19: 0.39999999999999997
Puntaje para k=20: 0.37142857142857144
El número de vecinos con el puntaje más alto es: 7


In [101]:
#Según el k_scores, podemos ver que a partir de 7 vecinos es totalmente irrelevante cuantos pongamos, 
#así que utilizaremos n_neighbors=7
#Definimos y entrenamos kNN

atributos_entrenamiento_2d = np.array(atributos_entrenamiento).reshape(-1, 1)
atributos_prueba_2d = np.array(atributos_prueba).reshape(-1, 1)
objetivos_entrenamiento_2d = np.array(objetivos_entrenamiento).reshape(-1, 1)
objetivos_prueba_2d = np.array(objetivos_prueba).reshape(-1, 1)

clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=7, metric='hamming')
clasif_kNN.fit(atributos_entrenamiento_2d, objetivos_entrenamiento_2d)

#Probamos la predicción con los atributos de prueba
print('Predicción kNN: ',clasif_kNN.predict(atributos_prueba_2d))
#Hacemos el score con kNN
print('Precisión kNN: ',clasif_kNN.score(atributos_prueba_2d, objetivos_prueba_2d))
#Hacemos el score con cross validation
print('Precisión cross validation: {}'.format(np.mean(cross_val_score(clasif_kNN, np.array(atributos).reshape(-1, 1), np.array(objetivos).reshape(-1, 1), cv=cv))))

Predicción kNN:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Precisión kNN:  0.4857142857142857
Precisión cross validation: nan


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode